# Part 1

In [1]:
# Stolen from day 6 and modified for today.
import itertools

class Matrix:
    ''' A dense matrix. '''
    def __init__(self, rows, cols, initial=' '):
        self._rows = rows
        self._cols = cols
        self._cells = [[initial] * cols for _ in range(rows)]
    
    def __repr__(self):
        return 'Matrix<{}x{}>'.format(self._rows, self._cols)
    
    def __str__(self):
        render = ''
        for row in self._cells:
            render += ''.join(str(v) for v in row) + '\n'
        return render

    def __getitem__(self, key):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        return self._cells[row][col]
    
    def __setitem__(self, key, val):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        self._cells[row][col] = val
    
    def clone(self):
        m = Matrix(self._rows, self._cols)
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            m[row,col] = self[row,col]
        return m

    def get(self, row, col, default=None):
        try:
            return self[row,col]
        except (IndexError,KeyError):
            return default
    
    @property
    def size(self):
        return self._rows, self._cols

    def values(self):
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield self[row, col]
    
    def visit(self):
        ''' Yield each cell and its coordinates. '''
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield row, col, self[row,col]

In [2]:
def build_map(depth, target_x, target_y, map_x, map_y):
    ''' Build a map of the given depth from 0,0 to map_x, map_y. '''
    # Create a map that stores erosion levels.
    erosion_levels = Matrix(map_x+1, map_y+1, initial=None)

    def geologic_index(x, y):
        nonlocal target_x, target_y
        if x == y == 0:
            gi = 0
        elif (x, y) == (target_x, target_y):
            gi = 0
        elif x == 0:
            gi = y * 48271
        elif y == 0:
            gi = x * 16807
        else:
            gi = (erosion_levels[x-1,y] * erosion_levels[x,y-1])
        return gi

    def erosion_level(x, y):
        return (geologic_index(x, y) + depth) % 20183

    # Fill in the origin and axes first.
    erosion_levels[0,0] = erosion_level(0, 0)
    for x in range(1, map_x+1):
        erosion_levels[x,0] = erosion_level(x, 0)
    for y in range(1, map_y+1):
        erosion_levels[0,y] = erosion_level(0, y)
    
    # Fill in the rest of the map.
    for x, y in itertools.product(range(1, map_x+1), range(1, map_y+1)):
        erosion_levels[x,y] = erosion_level(x,y)

    # Convert map from erosion levels to region types.
    regions = Matrix(*erosion_levels.size)
    for x, y, erosion_level in erosion_levels.visit():
        regions[x,y] = erosion_level % 3
    return regions

def render_regions(regions, target_x, target_y):
    # Note that the Matrix class uses r,c coordinates, which are flipped
    # from the x,y coordinates used in this problem. When visualizing the
    # matrix, we compute a transpose and display that instead, so that
    # our visuals match up with the examples in the problem.
    REGION_CHARS = ['.', '=', '|']
    rows, cols = regions.size
    mat = Matrix(cols, rows)
    for x, y, region in regions.visit():
        if (x,y) == (0,0):
            char = 'M'
        elif (x,y) == (target_x, target_y):
            char = 'T'
        else:
            char = REGION_CHARS[regions[x,y]]
        mat[y,x] = char
    return str(mat)

def print_regions(regions, target_x, target_y):
    print(render_regions(regions, target_x, target_y))

In [3]:
regions = build_map(510, 10, 10, 15, 15)
print_regions(regions, 10, 10)

M=.|=.|.|=.|=|=.
.|=|=|||..|.=...
.==|....||=..|==
=.|....|.==.|==.
=|..==...=.|==..
=||.=.=||=|=..|=
|.=.===|||..=..|
|..==||=.|==|===
.=..===..=|.|||.
.======|||=|=.|=
.===|=|===T===||
=|||...|==..|=.|
=.=|=.=..=.||==|
||=|=...|==.=|==
|=.=||===.|||===
||.|==.|.|.||=||



In [4]:
def sum_regions(mat, target_x, target_y):
    sum_ = 0
    for x, y, val in mat.visit():
        if x <= target_x and y <= target_y:
            sum_ += val
    return sum_

In [5]:
sum_regions(regions, 10, 10)

114

In [6]:
# This is the problem input:
regions = build_map(5616, 10, 785, 10, 785)

In [7]:
with open('output.txt', 'w') as output_:
    output_.write(render_regions(regions, 10, 785))

In [8]:
sum_regions(regions, 10, 785)

8681

# Part 2

Part 2 is totally different from part 1, and it is way more difficult. My general strategy here is to create a matrix where each cell stores the lowest cost path from the origin to that cell.

To accomplish this, I define a function that computes the cost for all neighbors of a given cell. The cost for each neighbor is equal to the cost of the current cell plus the cost of moving to that neighbor. If this cost is less than the neighbor's current cost, then the neighbor's cost is updated to this lower cost, and the neighbor is placed on a stack of cells to recompute. This algorithm completes when the stack is empty.

There are two complications: first, the cost varies depending on which tool is currently equipped. Therefore, instead of storing one cost at each cell, I'm going to store three costs: the cost for arriving at that cell with that particular tool equipped. The cost of moving between two cells depends on their respective regions. If the two regions are same, then the cost is +1 for the two gears that are allowed and NA for the gear that is not allowed.

If two regions are different, then one gear will be in common between both regions. That gear is +1. One gear is not allowed in the new region and stays NA. The third gear is not allowed in the old region but is allowed in the new region, so it is equal to the first gear + 7, because you could move to this region and then switch gear.

    Wet -> Rock      (C in common, N not allowed, T=C+7)
    Wet -> Narrow    (N in common, C not allowed, T=N+7)
    Rock -> Wet      (C in common, T not allowed, N=C+7)
    Rock -> Narrow   (T in common, C not allowed, N=T+7)
    Narrow -> Wet    (N in common, T not allowed, C=N+7)
    Narrow -> Rock   (T in common, N not allowed, C=T+7)

The second complication is that the map grows infinitely in the positive x and y directions. If you're currently adjacent to a destination and you need to switch tools, then it takes 8 minutes to move there. If you try to go around on a path that doesn't require changing tools, you cannot go more than 3 steps out of the way spaces without losing optimality.

    Source      → 1 → 2 → 3
      ↓                   ↓
    Destination ← 6 ← 5 ← 4

If you had to go 4 steps out of the way, it would be just as fast to change tools and move directly to the destination. For this reason, I think I need to compute a matrix that is 3 units larger in each direction.

In [9]:
from dataclasses import dataclass, field

@dataclass
class PathCost:
    torch: int = field(default=None)
    climbing: int = field(default=None)
    neither: int = field(default=None)

    def __repr__(self):
        ''' This representation is a bit cryptic, but its just compact enough to 
        display 10 columns on my laptop screen. '''
        fmt = '{:02d}'
        torch = 'NA' if self.torch is None else fmt.format(self.torch)
        climbing = 'NA' if self.climbing is None else fmt.format(self.climbing)
        neither = 'NA' if self.neither is None else fmt.format(self.neither)
        return '{}{}{}'.format(torch, climbing, neither)
    
    def update(self, new_cost):
        ''' Copy any values in new cost that are less than the corresponding 
        values in this cost. Return True if any of the old values were modified. '''
        updated = False
        if new_cost.torch is not None and new_cost.torch < self.torch:
            self.torch = new_cost.torch
            updated = True
        if new_cost.climbing is not None and new_cost.climbing < self.climbing:
            self.climbing = new_cost.climbing
            updated = True
        if new_cost.neither is not None and new_cost.neither < self.neither:
            self.neither = new_cost.neither
            updated = True
        return updated

In [10]:
def get_move_cost(start_cost, start_region, new_region):
    ''' Return the cost associated with moving to a new region. See discussion
    above. '''
    if new_region == 0:
        if start_region == 0:
            # Rock -> Rock (C & T allowed, N not allowed)
            return PathCost(
                torch=start_cost.torch + 1,
                climbing=start_cost.climbing + 1,
                neither=None)
        elif start_region == 1:
            # Wet -> Rock (C in common, N not allowed, T=C+7)
            return PathCost(
                torch=start_cost.climbing + 8,
                climbing=start_cost.climbing + 1,
                neither=None)
        elif start_region == 2:
            # Narrow -> Rock (T in common, N not allowed, C=T+7)
            return PathCost(
                torch=start_cost.torch + 1,
                climbing=start_cost.torch + 8,
                neither=None)
    elif new_region == 1:
        if start_region == 0:
            # Rock -> Wet (C in common, T not allowed, N=C+7)
            return PathCost(
                torch=None,
                climbing=start_cost.climbing + 1,
                neither=start_cost.climbing + 8)
        elif start_region == 1:
            # Wet -> Wet (C & N allowed, T not allowed)
            return PathCost(
                torch=None,
                climbing=start_cost.climbing+1,
                neither=start_cost.neither+1)
        elif start_region == 2:
            # Narrow -> Wet (N in common, T not allowed, C=N+7)
            return PathCost(
                torch=None,
                climbing=start_cost.neither + 8,
                neither=start_cost.neither + 1)
    elif new_region == 2:
        if start_region == 0:
            # Rock -> Narrow (T in common, C not allowed, N=T+7)
            return PathCost(
                torch=start_cost.torch + 1,
                climbing=None,
                neither=start_cost.torch + 8)
        elif start_region == 1:
            # Wet -> Narrow (N in common, C not allowed, T=N+7)
            return PathCost(
                torch=start_cost.neither + 8,
                climbing=None,
                neither=start_cost.neither + 1)
        elif start_region == 2:
            # Narrow -> Narrow (T & N allowed, C not allowed)
            return PathCost(
                torch=start_cost.torch+1,
                climbing=None,
                neither=start_cost.neither+1)
    raise Exception('Should not happen')

In [11]:
# Moving from rock->rock. Torch and climbing should both go up by 1,
# and neither should be NA.
get_move_cost(PathCost(torch=0, climbing=0, neither=None), 0, 0)

0101NA

In [12]:
# Moving from rock->wet, climbing should go up 1 because it is allowed
# in both regions. Torch should be NA because it is not allowed in wet.
# Neither should be 8, because you can take climbing gear (1 minute) 
# and then switch to neither (7 minutes).
get_move_cost(PathCost(torch=0, climbing=0, neither=None), 0, 1)

NA0108

In [35]:
def shortest_path(regions, target_x, target_y, debug=False):
    ''' Compute the cost of the shortest path through the regions matrix
    to reach the target coordinates. '''
    # The cost at the origin is zero when you have the torch. If you switch
    # equipment right there, then you incur cost before you move.
    costs = Matrix(*regions.size, initial=None)
    costs[0,0] = PathCost(torch=0, climbing=7, neither=None)
    map_x, map_y = regions.size

    # Keep a set of cells that need to be processed.
    to_process = set()
    to_process.add((0,0))
    while to_process:
        # Get the next cell to process.
        start_x, start_y = to_process.pop()
        start_cost = costs[start_x, start_y]
        start_region = regions[start_x, start_y]
        
        # Check the 4 neighbors
        neighbors = (
            (start_x-1,start_y  ),
            (start_x+1,start_y  ),
            (start_x  ,start_y-1),
            (start_x  ,start_y+1),
        )
        for neighbor_x, neighbor_y in neighbors:
            if neighbor_x < 0 or neighbor_y < 0 or \
               neighbor_x >= map_x or neighbor_y >= map_y:
                continue
            neighbor_region = regions[neighbor_x, neighbor_y]
            old_cost = costs[neighbor_x, neighbor_y]
            new_cost = get_move_cost(start_cost, start_region, neighbor_region)
            if old_cost is None:
                # If there is no previous cost computed for this neighbor, then this cost
                # is the new optimal cost.
                costs[neighbor_x, neighbor_y] = new_cost
                to_process.add((neighbor_x, neighbor_y))
            else:
                # If any of the the new costs are lower than the old costs, then
                # update the cost and put the neighbor on the stack.
                if old_cost.update(new_cost):
                    to_process.add((neighbor_x, neighbor_y))
    if debug:
        print_costs(costs, regions)
    return costs[target_x,target_y].torch

def print_costs(costs, regions):
    ''' For debugging. '''
    REGION_CHARS = ['.', '=', '|']
    cx, cy = costs.size
    print('   {}'.format(' '.join('{:7d}'.format(x) for x in range(cx))))
    for y in range(cy):
        line = ' '.join(REGION_CHARS[regions[x,y]] + str(costs[x,y] if costs[x,y] is not None else 'NA')
            for x in range(cx))
        print('{:2d} {}'.format(y, line))

In [14]:
# Sample problem. Note that I'm building regions with 3 extra units in each dimension:
# 13 instead of 10.
regions = build_map(510, 10, 10, 13, 13)
print_regions(regions, 10, 10)

M=.|=.|.|=.|=|
.|=|=|||..|.=.
.==|....||=..|
=.|....|.==.|=
=|..==...=.|==
=||.=.=||=|=..
|.=.===|||..=.
|..==||=.|==|=
.=..===..=|.||
.======|||=|=.
.===|=|===T===
=|||...|==..|=
=.=|=.=..=.||=
||=|=...|==.=|



In [15]:
shortest_path(regions, 10, 10, debug=True)

         0       1       2       3       4       5       6       7       8       9      10      11      12      13
 0 .0007NA =NA0810 .1609NA |17NA12 =NA2013 .2121NA |22NA15 .2330NA |24NA31 =NA3232 .2633NA |27NA34 =NA3435 |31NA36
 1 .0108NA |02NA09 =NA1010 |18NA11 =NA1912 |20NA13 |21NA14 |22NA15 .2330NA .2431NA |25NA32 .2632NA =NA3336 .3034NA
 2 .0209NA =NA1010 =NA1111 |19NA12 .2020NA .2121NA .2222NA .2323NA |24NA31 |25NA32 =NA3033 .2731NA .2832NA |29NA36
 3 =NA1017 .1811NA |19NA12 .2022NA .2121NA .2222NA .2323NA |24NA31 .2527NA =NA2833 =NA2934 .2830NA |29NA36 =NA4337
 4 =NA1118 |19NA19 .2024NA .2123NA =NA2229 =NA2330 .2424NA .2525NA .2626NA =NA2734 .3028NA |29NA36 =NA4137 =NA4238
 5 =NA1219 |20NA20 |21NA21 .2224NA =NA2330 .3124NA =NA2532 |26NA33 |27NA34 =NA2835 |31NA36 =NA3937 .4740NA .4841NA
 6 |22NA20 .2127NA =NA2622 .2325NA =NA2431 =NA2532 =NA2633 |27NA34 |28NA35 |29NA36 .3037NA .3138NA =NA3941 .4740NA
 7 |23NA21 .2228NA .2327NA =NA2633 =NA2532 |40NA33 |41NA34 =NA3035 .2931NA |30NA

45

In [16]:
# This is the problem input. Again, I'm building 3 extra units in each dimension.
regions = build_map(5616, 10, 785, 13, 788)

In [17]:
%%time
# This is too high!
print(shortest_path(regions, 10, 785))

CPU times: user 14.9 s, sys: 31.2 ms, total: 14.9 s
Wall time: 14.9 s


1130

In [37]:
# Let's try a bigger regions map.
regions = build_map(5616, 10, 785, 15, 790)

In [38]:
%%time
# 1122 is still too high! But clearly my reasoning about making a map
# that is 3 units larger is faulty: this map is 5 units larger than the target,
# and it has a smaller total than the previous map. I will try making larger
# and larger maps until the path stops decreasing.
shortest_path(regions, 10, 785)

CPU times: user 16.7 s, sys: 16.7 ms, total: 16.7 s
Wall time: 16.7 s


1122

In [39]:
regions = build_map(5616, 10, 785, 20, 800)

In [40]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 24.1 s, sys: 48.1 ms, total: 24.2 s
Wall time: 24.2 s


1108

In [41]:
regions = build_map(5616, 10, 785, 25, 810)

In [42]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 30.8 s, sys: 17.6 ms, total: 30.8 s
Wall time: 30.8 s


1088

In [43]:
regions = build_map(5616, 10, 785, 30, 820)

In [45]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 47.3 s, sys: 63.3 ms, total: 47.4 s
Wall time: 47.5 s


1075

In [46]:
regions = build_map(5616, 10, 785, 35, 830)

In [47]:
%%time
# Got the same answer twice! But it still says my answer is too high.
shortest_path(regions, 10, 785)

CPU times: user 1min, sys: 76.1 ms, total: 1min 1s
Wall time: 1min 1s


1075

In [48]:
regions = build_map(5616, 10, 785, 40, 840)

In [49]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 1min 10s, sys: 81.8 ms, total: 1min 10s
Wall time: 1min 10s


1075

In [50]:
regions = build_map(5616, 10, 785, 50, 860)

In [51]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 1min 22s, sys: 74.6 ms, total: 1min 22s
Wall time: 1min 22s


1070

In [52]:
regions = build_map(5616, 10, 785, 60, 880)

In [53]:
%%time
shortest_path(regions, 10, 785)

CPU times: user 1min 42s, sys: 153 ms, total: 1min 42s
Wall time: 1min 42s


1070

In [54]:
regions = build_map(5616, 10, 785, 70, 900)

In [55]:
%%time
# I got 1070 several times in a row, so check if its the right answer...
# YES!
shortest_path(regions, 10, 785)

CPU times: user 2min, sys: 155 ms, total: 2min 1s
Wall time: 2min 1s


1070